# Imports

In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Deep learning with TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet18, ResNet18_Weights

# Natural Language Processing (NLP)
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Miscellaneous
import os
import re
import time
import pickle
from PIL import Image

from google.colab import drive
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


# Data Import

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

data = torch.load("/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman.pt")
train_length = (int)(0.9 * len(data))
train_set, val_set = torch.utils.data.random_split(data, [train_length, len(data) - train_length])

train_loader = DataLoader(train_set, shuffle = True, pin_memory = True, num_workers = 2)
test_loader = DataLoader(val_set, shuffle = False, pin_memory = True, num_workers = 2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import os.path

frame_process = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

def crop(image):
  return image.crop((80, 58, 577, 428))

spect_process = transforms.Compose([
    transforms.Lambda(crop),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

frame_directory = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_split/"
spect_directory = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_spectro/"
folders = ["anger/", "disgust/", "fear/", "joy/", "sadness/", "surprise/"]
df = pd.read_csv("/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/ekman6_texts/en_transcripts.csv")

def preprocess_text(text: str) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    # 1. tokenize
    tokens = nltk.word_tokenize(text)
    # 2. check if stopword
    tokens = [w.lower() for w in tokens if not w in stopwords.words("english")]
    return tokens

def get_frame_tensor(i):
  frames = []
  last_valid_filename = ""
  for j in range(1, 30):
    filename = str(i) + "_" + str(j) + ".jpg"
    if not os.path.isfile(frame_directory + folders[(int)(i/50)] + filename):
      filename = last_valid_filename
    if filename == "":
      print(str(i) + "_" + str(j) + ".jpg")
    file = Image.open(frame_directory + folders[(int)(i/50)] + filename)
    file = frame_process(file)
    frames.append(file)
    last_valid_filename = filename
  frames = torch.stack(frames)
  return frames

def get_spect_tensor(i):
  filename = str(i) + ".jpg"
  file = Image.open(spect_directory + folders[(int)(i/50)] + filename)
  file = spect_process(file)
  return file

dictionary = {
    'EMPTY': 1
}

def get_text_tensor(i):
  text = df.columns[i]
  text = preprocess_text(text)
  liszt = []
  for i in range(len(text)):
    if text[i] in dictionary:
      liszt.append((int)(dictionary[text[i]]))
    else:
      size = len(dictionary) + 1
      dictionary[text[i]] = size
      liszt.append((int)(dictionary[text[i]]))
  return torch.Tensor(liszt).to(device).to(torch.int64), len(text)


In [8]:
text_tensors = []
text_lengths = []
MXLEN = 0

num_input_videos = 300

for i in range(num_input_videos):
  tt, text_length = get_text_tensor(i)
  text_tensors.append(tt)
  text_lengths.append(text_length)
  MXLEN = max(MXLEN, (int)(tt.size(0)))

text_tensors = torch.nn.utils.rnn.pad_sequence(text_tensors, batch_first = True)

In [9]:
#frame_tensors = []
spect_tensors = []

for i in range(300):
  #frame_tensors.append(get_frame_tensor(i))
  spect_tensors.append(get_spect_tensor(i))

# Models

In [ ]:
class LSTM_Classifier(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
    super(LSTM_Classifier, self).__init__()
    # Embedding layer converts integer sequences to vector sequences
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    # LSTM layer process the vector sequences
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional, dropout = dropout, batch_first = True)
    #                     input           output                             bisexual?                      dropout_prob      whether or not batch_size comes first in the tensor.size()
    # Dense layer to predict
    self.fc = nn.Linear(hidden_dim * (2 if bidirectional == True else 1), output_dim)
    # Prediction activation function
    self.sigmoid = nn.Sigmoid()

  def forward(self, text, text_lengths):
    embedded = self.embedding(text) # embedded version
    # Thanks to packing, LSTM don't see padding tokens
    # and this makes our model better
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True, enforce_sorted = False)
    packed_output, (hidden_state, cell_state) = self.lstm(packed_embedded)
    # Concatenating the final forward and backward hidden states
    hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)
    dense_outputs = self.fc(hidden)
    #Final activation function
    dense_outputs = nn.functional.relu(dense_outputs)
    return dense_outputs

In [ ]:
class Smash(nn.Module):
  def __init__(self):
    super(Smash, self).__init__()
    self.fcnn = nn.Sequential(
      nn.Linear(12, 12),
      nn.ReLU(),
      nn.Linear(12, 6)
    )
    self.bilstm = LSTM_Classifier(vocab_size = len(dictionary) + 1,
                         embedding_dim = 100,
                         hidden_dim = 64,
                         output_dim = 6,
                         n_layers = 5,
                         bidirectional = True,
                         dropout = 0.5).to(device)
    self.resn = resnet18(num_classes = 6)

  def forward(self, x1, x2):
    x1 = self.resn(x1)
    x2 = self.bilstm(x2)
    x1 = torch.cat((x1, x2), dim = -1)
    x1 = self.fcnn(x1)
    return x1

# Training Loop

In [27]:
model2 = resnet18(pretrained = True).to(device) # output == 1000 neurons (when pretrained = True)
model2.fc = nn.Linear(512, 6).to(device) # modifying the output to 6 neurons

model3 = LSTM_Classifier(vocab_size = len(dictionary) + 1,
                         embedding_dim = 100,
                         hidden_dim = 64,
                         output_dim = 6,
                         n_layers = 5,
                         bidirectional = True,
                         dropout = 0.5).to(device)

model4 = Smash().to(device)

epochs = 2

optimizer2 = optim.Adam(model2.parameters(), lr = 0.01)
optimizer3 = optim.Adam(model3.parameters(), lr = 0.01)
optimizer4 = optim.Adam(model4.parameters(), lr = 0.01)
loss_fn = nn.CrossEntropyLoss()

tl = []

def train():
  model2.train()
  model3.train()
  model4.train()

def eval():
  model2.eval()
  model3.eval()
  model4.eval()

def zerograd():
  optimizer2.zero_grad()
  optimizer3.zero_grad()
  optimizer4.zero_grad()

def step():
  optimizer2.step()
  optimizer3.step()
  optimizer4.step()

for i in range(epochs):
  train()
  train_loss = 0.00
  correct = 0.00
  total = 0.00
  for index, label in train_loader:
    index = index.cuda()
    label = label.cuda()

    # getting the text for the index-th video
    text = text_tensors[index].to(device)

    # if the text ends up being nothing
    if text_lengths[index] == 0:
      text = torch.IntTensor([dictionary['EMPTY']]).to(device)

      # dimensionality voodoo
      text = text.unsqueeze(0)
      text = text.unsqueeze(0)
      text_lengths[index] = 1

    text = text.squeeze(1).to(torch.float32)

    # getting the spectrogram
    spect = spect_tensors[index].to(device)
    spect = spect.unsqueeze(0)
    spect.requires_grad = True
    assert spect.size(0) == 1

    # zero-ing gradients
    zerograd()

    # output of ResNet on the spectrogram
    y_spect = model2(spect).to(device)

    # output of the bi-LSTM on the text
    y_text = model3(text, [text_lengths[index]]).to(device)

    # adding all of the tensors into a 'super-tensor'
    input = torch.cat((y_spect, y_text), dim=-1)
    input.requires_grad_(True)

    # output of the final NN on the super-tensor
    y_pred = model4(input).to(device)

    # creating one-hot vector for the label for the index-th video
    labels = torch.full((1, 6,), 0.00).cuda()
    labels[0][label] = 1.00

    # computing the Cross Entropy Loss and backpropagating
    loss = loss_fn(y_pred, labels)
    loss.backward()

    #soft_pred = torch.nn.functional.log_softmax(y_pred, dim=0)
    print(y_pred, labels)
    print(loss.item())
    print(model4.fcnn[2].weight.grad)

    # updating gradients
    step()

    # statistics
    train_loss += loss.item()/len(train_loader)
    prediction = y_pred.argmax(dim=1)
    correct += (prediction.eq(label).sum()).item()
    total += label.size(0)

  # more statistics
  tl.append(train_loss)
  print(f"Epoch: {i+1}/{epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {correct/total:.4f}")

tensor([[-0.0421,  0.0296, -0.2133, -0.1826, -0.1725, -0.0940]],
       device='cuda:0', grad_fn=<AddmmBackward0>) tensor([[0., 0., 1., 0., 0., 0.]], device='cuda:0')
1.89628005027771
tensor([[ 0.0000,  0.0161,  0.0021,  0.0637,  0.0380,  0.0014,  0.0089,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0581],
        [ 0.0000,  0.0172,  0.0023,  0.0684,  0.0408,  0.0016,  0.0095,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0625],
        [-0.0000, -0.0766, -0.0100, -0.3038, -0.1813, -0.0069, -0.0422, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.2774],
        [ 0.0000,  0.0140,  0.0018,  0.0553,  0.0330,  0.0013,  0.0077,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0505],
        [ 0.0000,  0.0141,  0.0018,  0.0559,  0.0334,  0.0013,  0.0078,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0510],
        [ 0.0000,  0.0152,  0.0020,  0.0605,  0.0361,  0.0014,  0.0084,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0552]], device='cuda:0')
tensor([[-0.7458, -0.4587,  0.6562, 

KeyboardInterrupt: 

In [17]:
print((model3.lstm._parameters['weight_ih_l0'].grad == 0.0).all())

tensor(True, device='cuda:0')


In [18]:
print((model2.conv1.weight.grad == 0.0).all())

tensor(True, device='cuda:0')


In [19]:
print((model4.fcnn[0].weight.grad == 0.0).all())

tensor(True, device='cuda:0')
